In [1]:
# various import statements
import os
import scanpy as sc
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import re
import scipy
import sys
import gc
sys.path.append('/home/matthew.schmitz/utils/mts-utils/')
from genomics import sc_analysis


In [2]:
cs_path='/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/Team/Matthew/data/cortex_data/jorstad_cross_species/'
rfiles=os.listdir(cs_path)
adatas={}
for f in rfiles:
    print(f)
    if not f in adatas.keys() and '.h5ad' in f:
        adata=sc.read_h5ad(os.path.join(cs_path,f))
        adata.obs['file']=f
        adatas[f]=adata

Callithrix_jacchus-mtg.h5ad
Pan_troglodytes-mtg.h5ad
Macaca_mulatta-mtg.h5ad
Gorilla_gorilla-mtg.h5ad
Homo_sapiens-mtg.h5ad


In [3]:
ca_path='/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/Team/Matthew/data/cortex_data/jorstad_cross_areal/'
rfiles=os.listdir(ca_path)
for f in rfiles:
    print(f)
    if not f in adatas.keys() and '.h5ad' in f:
        adata=sc.read_h5ad(os.path.join(ca_path,f))
        adata.obs['file']=f
        adatas[f]=adata

Homo_sapiens-ang.h5ad
MTG_MAPPING
Homo_sapiens-m1.h5ad
Homo_sapiens-acc.h5ad
Homo_sapiens-dfc.h5ad
Homo_sapiens-v1.h5ad
Homo_sapiens-mtg_Hodge_MAPPING
Homo_sapiens-glia.h5ad
mtg-crossarea_mapping_reference
MTG_Hodge_MAPPING
cell_types
Homo_sapiens-s1.h5ad
Homo_sapiens-mtg.h5ad
mtg_mapping_reference
smart_seq.h5ad
Homo_sapiens-a1.h5ad
v1_mapping_reference


In [4]:
del adatas['smart_seq.h5ad']

In [5]:
columns=[]
for k in adatas.keys():
    columns+=list(adatas[k].obs.columns)
columns=list(set(columns))

In [6]:
for k in adatas.keys():
    adatas[k].obs.loc[:,~adatas[k].obs.columns.isin(columns)]='nan'


In [7]:
for k in adatas.keys():
    #print(adatas[k].var['feature_name'])
    adatas[k].var.index=adatas[k].var['feature_name']

In [8]:
adata=sc.concat(adatas.values(),join='inner',merge='unique',uns_merge='unique')

/home/matthew.schmitz/Matthew/utils/miniforge3/envs/scanpy/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [9]:
del adatas

In [10]:
adata=adata[adata.obs['assay']=="10x 3' v3",:]

In [11]:
adata.obs['batch']=adata.obs.index.to_series().str.replace("A|T|C|G","",regex=True)

/scratch/fast/15734288/ipykernel_297884/1602239871.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['batch']=adata.obs.index.to_series().str.replace("A|T|C|G","",regex=True)
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/scanpy/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/matthew.schmitz/Matthew/utils/miniforge3/envs/scanpy/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [17]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           1.0T        207G        509G        179M        290G        797G
Swap:          4.0G        315M        3.7G


In [18]:
gc.collect()

2666

In [19]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           1.0T        207G        509G        179M        290G        797G
Swap:          4.0G        315M        3.7G


In [15]:
adata.write_h5ad('/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/Team/Matthew/data/cortex_data/jorstad_merge_all.h5ad')

In [16]:
#adata=sc.read_h5ad('/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/Team/Matthew/data/cortex_data/jorstad_merge_all.h5ad')

KeyboardInterrupt: 

In [ ]:
len(adata.obs.index.to_series().str.replace("A|T|C|G","",regex=True).unique())

In [20]:
adata.X.data=np.round(2.**adata.X.data-1)
adata.X=adata.X.astype(int)
adata.layers['UMIs']=adata.X

In [21]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [22]:
#adata.layers['UMIs']=adata.X.copy()
#adata.layers['UMIs'].data=np.round(2.**adata.layers['UMIs'].data-1)
#adata.layers['UMIs']=adata.layers['UMIs'].astype(int)

In [23]:
adata.write_h5ad('/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/Team/Matthew/data/cortex_data/jorstad_merge_all.h5ad')

In [24]:
adata

AnnData object with n_obs × n_vars = 1667831 × 14322
    obs: 'nCount_RNA', 'nFeature_RNA', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'organism_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'tissue_ontology_term_id', 'is_primary_data', 'donor_id', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'file', 'batch'
    var: 'feature_is_filtered', 'feature_reference', 'feature_biotype', 'ensembl_id'
    uns: 'batch_condition', 'schema_reference', 'CrossArea_subclass_colors', 'Donor_colors', 'Region_colors', 'WithinArea_subclass_colors', 'log1p'
    layers: 'UMIs'

In [25]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           1.0T        308G        260G        180M        438G        696G
Swap:          4.0G        315M        3.7G


In [26]:
adata.layers['UMIs'].max()

5307